In [6]:
import aitoolbox.context as aictx

context = aictx.Context.get()
sources = context.get_sources()
sources.set("prompt","What is the ultimate answer to life, the universe and everything?")

# Setup

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [ ]:
TOKEN="hZmAMwYmrG7Xvxsg8KNmLxZ9TWWJvA"

In [ ]:
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

# Service

In [7]:
import aitoolbox.errors as aierr
if aictx.Context.get().get_sources().get("token") != TOKEN:
    raise aierr.ServiceError("Token authorization failed")

In [ ]:
prompt=aictx.Context.get().get_sources().get("prompt")
print(prompt)

In [ ]:
in_tokens = tokenizer(prompt)
sequences = pipeline(
    prompt,
    max_length=500,
    penalty_alpha=0.8,
    top_k=20,
    return_full_text=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
resp = sequences[0]["generated_text"]
out_tokens = tokenizer(resp)

In [ ]:
print(resp)

In [ ]:
aictx.Context.get().get_destinations().set("resp",resp)